In [2]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from xgboost.sklearn import XGBClassifier

In [3]:
df = pd.read_json('data/diabetes/diabetes.json')
df_mod = df.drop(columns=['SkinThickness','Insulin'], axis=1)
df_mod.head()


,DiabetesPedigreeFunction,BloodPressure,Pregnancies,BMI,Outcome,Age,Glucose
0,0.627,72,6,33.6,1,50,148
1,0.351,66,1,26.6,0,31,85
2,0.672,64,8,23.3,1,32,183
3,0.167,66,1,28.1,0,21,89
4,2.288,40,0,43.1,1,33,137


In [4]:
features = df_mod[['DiabetesPedigreeFunction','BloodPressure','Pregnancies','BMI','Age', 'Glucose']]
outputs = df[['Outcome']]

outputs = outputs.astype('float64')
features = features.astype('float64')
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   DiabetesPedigreeFunction  696 non-null    float64
 1   BloodPressure             696 non-null    float64
 2   Pregnancies               696 non-null    float64
 3   BMI                       696 non-null    float64
 4   Age                       696 non-null    float64
 5   Glucose                   696 non-null    float64
dtypes: float64(6)
memory usage: 32.8 KB


In [6]:
sc = StandardScaler()
features_norm = sc.fit_transform(features)
outputs_mod = outputs.to_numpy().flatten()
x_train, x_test, y_train, y_test = train_test_split(features_norm, outputs_mod)

In [7]:
xgb = XGBClassifier(eval_metric=["error"])
x_train.shape



(522, 6)

In [8]:
y_train.shape

(522,)

In [9]:
xgb.fit(x_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=['error'],
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [10]:
y_pred = xgb.predict(x_test)
k_fold = KFold(n_splits=10, shuffle=True)
score = cross_val_score(xgb, x_train, y_train, cv=k_fold, scoring='accuracy')
print('Mean:', score.mean(),'Std:', score.std())

Mean: 0.729934687953556 Std: 0.051497634790501436
